In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HH9ECnEeBwB663gIBlcY")
project = rf.workspace("mazen-alwaqdei-r4iyg").project("plate-license-dvt6r")
version = project.version(5)
dataset = version.download("tensorflow")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.1

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to plate-license-5 in tensorflow:: 100%|██████████| 4047/4047 [00:00<00:00, 4425.18it/s]


# Data Preprocessing

In [ ]:
import os
import numpy as np
import pandas as pd
# Define the path to the folder containing images and corresponding .txt files
folder_path_train = "/content/plate-license-5/train/"
folder_path_test = "/content/plate-license-5/test/"
df = pd.read_csv('/content/plate-license-5/train/_annotations.csv')
df_test = pd.read_csv('/content/plate-license-5/test/_annotations.csv')
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,36auaeejilka1_jpg.rf.002cacd4aa2ffc5407b7e20fd...,640,640,LicensePlate,286,288,385,356
1,8i4tjjo711aa1_jpg.rf.004a4953e5f74d93227d82c15...,640,640,LicensePlate,239,331,308,377
2,syn_2819_jpg.rf.009ecfa0ffd1e95aa5ea329d9f201c...,640,640,X,50,76,107,605
3,syn_2819_jpg.rf.009ecfa0ffd1e95aa5ea329d9f201c...,640,640,B,120,64,187,570
4,syn_2819_jpg.rf.009ecfa0ffd1e95aa5ea329d9f201c...,640,640,R,195,53,258,565


In [ ]:
# import os
# import numpy as np
# import pandas as pd
# # Define the path to the folder containing images and corresponding .txt files
# folder_path_train = "/content/VEHICLE-CLASSIFICATION-11/train/"
# folder_path_test = "/content/VEHICLE-CLASSIFICATION-11/test/"
# df = pd.read_csv('/content/VEHICLE-CLASSIFICATION-11/train/_annotations.csv')
# df_test = pd.read_csv('/content/VEHICLE-CLASSIFICATION-11/test/_annotations.csv')
# df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,04142_jpg.rf.60c4a33469a29c11a7244ef0369e5a33.jpg,260,194,suv,25,38,209,170
1,06209_jpg.rf.60c5f510889740f7feaaa7f2977835bf.jpg,640,480,pickup,15,205,632,471
2,01222_jpg.rf.60c99dc6d37915540106dd40d6dd51de.jpg,640,480,van,19,65,596,347
3,07117_jpg.rf.60d6cb04a7d6c0da5e775cfa6e7cd925.jpg,424,360,sedan,15,90,417,281
4,05243_jpg.rf.60e3bcaae25e2904694bf671cb8a9d92.jpg,112,84,van,2,15,110,83


In [ ]:
# import cv2
# import os
# data = []
# for filename in os.listdir(folder_path_train):
#     if filename.endswith('.jpg'):
#         image_path = os.path.join(folder_path_train, filename)
#         image = cv2.imread(image_path)  #Reading the image
#         data.append(image)

# import numpy as np
# for i in range(len(data)):
#     data[i] = cv2.resize(data[i], (128, 128))

# data_scaled_np = np.array(data)



In [ ]:
df_reduced  =  df[['filename','xmin','ymin','xmax','ymax']]
df_reduced.head()

,filename,xmin,ymin,xmax,ymax
0,36auaeejilka1_jpg.rf.002cacd4aa2ffc5407b7e20fd...,286,288,385,356
1,8i4tjjo711aa1_jpg.rf.004a4953e5f74d93227d82c15...,239,331,308,377
2,syn_2819_jpg.rf.009ecfa0ffd1e95aa5ea329d9f201c...,50,76,107,605
3,syn_2819_jpg.rf.009ecfa0ffd1e95aa5ea329d9f201c...,120,64,187,570
4,syn_2819_jpg.rf.009ecfa0ffd1e95aa5ea329d9f201c...,195,53,258,565


In [ ]:
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

# Assuming you have a DataFrame named 'df' with columns: filename, xmin, ymin, xmax, ymax
# You can replace 'df' with your actual DataFrame name

# Initialize the list of data (images), target output predictions
# (bounding box coordinates), and filenames of individual images
data = []
targets = []
filenames = []

for index, row in df_reduced.iterrows():
    filename = row['filename']
    startX, startY, endX, endY = row['xmin'], row['ymin'], row['xmax'], row['ymax']

    # Derive the path to the input image (replace 'folder_path_train' with your actual path)
    imagePath = folder_path_train + filename

    # Load the image (in OpenCV format) and grab its dimensions
    image = cv2.imread(imagePath)
    h, w = 640.0, 640.0  # Set your desired dimensions

    # Scale the bounding box coordinates relative to the spatial dimensions of the input image
    startX = float(startX) / w
    startY = float(startY) / h
    endX = float(endX) / w
    endY = float(endY) / h

    # Load the image and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)

    # Update our lists
    data.append(image)
    targets.append((startX, startY, endX, endY))
    filenames.append(filename)


In [ ]:
len(data)

10490

In [ ]:
len(targets)

10490

In [ ]:
data[0].shape

(224, 224, 3)

In [ ]:
data_np = np.array(data)
targets_np = np.array(targets)

# Prepairing test data

In [ ]:
df_reduced_test  =  df_test[['filename','xmin','ymin','xmax','ymax']]
df_reduced_test.head()

,filename,xmin,ymin,xmax,ymax
0,syn_656_jpg.rf.0d7ebca07aca585c73361a1f33e8d70...,103,56,181,290
1,syn_656_jpg.rf.0d7ebca07aca585c73361a1f33e8d70...,216,62,293,293
2,syn_656_jpg.rf.0d7ebca07aca585c73361a1f33e8d70...,373,63,463,292
3,syn_656_jpg.rf.0d7ebca07aca585c73361a1f33e8d70...,483,58,565,296
4,syn_656_jpg.rf.0d7ebca07aca585c73361a1f33e8d70...,135,358,223,597


In [ ]:
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

# Assuming you have a DataFrame named 'df' with columns: filename, xmin, ymin, xmax, ymax
# You can replace 'df' with your actual DataFrame name

# Initialize the list of data (images), target output predictions
# (bounding box coordinates), and filenames of individual images
data_test = []
targets_test = []
filenames = []

for index, row in df_reduced_test.iterrows():
    filename = row['filename']
    startX, startY, endX, endY = row['xmin'], row['ymin'], row['xmax'], row['ymax']

    # Derive the path to the input image (replace 'folder_path_train' with your actual path)
    imagePath = folder_path_test + filename

    # Load the image (in OpenCV format) and grab its dimensions
    image = cv2.imread(imagePath)
    h, w = 640.0, 640.0  # Set your desired dimensions

    # Scale the bounding box coordinates relative to the spatial dimensions of the input image
    startX = float(startX) / w
    startY = float(startY) / h
    endX = float(endX) / w
    endY = float(endY) / h

    # Load the image and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)

    # Update our lists
    data_test.append(image)
    targets_test.append((startX, startY, endX, endY))
    filenames.append(filename)


In [ ]:
X_test = np.array(data_test)
y_test = np.array(targets_test)

# CNN

In [ ]:
data_np.shape

(15264, 224, 224, 3)

In [ ]:
targets_np.shape

(15264, 4)

In [ ]:
import tensorflow as tf
from tensorflow import keras
model_cnn = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=75, kernel_size=3, activation='relu', padding = 'same'),
    keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    # keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(4)
])
# Compile the model
model_cnn.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import time


Time_cnn = time.time()
early_stopping = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)

model_cnn.fit(data_np, targets_np,
              epochs=1000,
              callbacks=[early_stopping],
              batch_size=512)

Time_cnn = time.time() - Time_cnn


Epoch 1/1000
30/30 [==============================] - 55s 1s/step - loss: 42136.1328 - accuracy: 0.2158
Epoch 2/1000
30/30 [==============================] - 15s 504ms/step - loss: 0.3143 - accuracy: 0.1016
Epoch 3/1000
30/30 [==============================] - 15s 503ms/step - loss: 0.1449 - accuracy: 0.5867
Epoch 4/1000
30/30 [==============================] - 15s 505ms/step - loss: 0.0854 - accuracy: 0.6743
Epoch 5/1000
30/30 [==============================] - 15s 506ms/step - loss: 0.0729 - accuracy: 0.6927
Epoch 6/1000
30/30 [==============================] - 15s 508ms/step - loss: 0.0659 - accuracy: 0.6986
Epoch 7/1000
30/30 [==============================] - 15s 507ms/step - loss: 0.0602 - accuracy: 0.7049
Epoch 8/1000
30/30 [==============================] - 15s 510ms/step - loss: 0.0569 - accuracy: 0.7087
Epoch 9/1000
30/30 [==============================] - 15s 509ms/step - loss: 0.0551 - accuracy: 0.7113
Epoch 10/1000
30/30 [==============================] - 15s 511ms/step - 

In [ ]:
print(  f"Time_cnn: {Time_cnn}")

Time_cnn: 451.8011953830719


In [ ]:
loss_cnn, accuracy_cnn = model_cnn.evaluate(X_test, y_test)

68/68 [==============================] - 2s 21ms/step - loss: 0.0548 - accuracy: 0.7065


In [ ]:
model_cnn.save('cnn_number_plate_regressor.keras')

# CNN + LSTM

In [ ]:
import tensorflow as tf
from tensorflow import keras

model_cnn_lstm = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=75, kernel_size=3, activation='relu', padding = 'same'),
    keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Reshape((-1, 64)),  # Reshape the output from Conv2D to be 3D as required by LSTM
    keras.layers.LSTM(64),  # Add LSTM layer with 64 units
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(4)
])

# Compile the model
model_cnn_lstm.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import time


Time_cnn_lstm = time.time()
early_stopping = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)

model_cnn_lstm.fit(data_np, targets_np,
              epochs=1000,
              callbacks=[early_stopping],
              batch_size=512)

Time_cnn_lstm = time.time() - Time_cnn_lstm


Epoch 1/1000
30/30 [==============================] - 19s 551ms/step - loss: 0.0890 - accuracy: 0.6557
Epoch 2/1000
30/30 [==============================] - 16s 550ms/step - loss: 0.0490 - accuracy: 0.7058
Epoch 3/1000
30/30 [==============================] - 16s 549ms/step - loss: 0.0469 - accuracy: 0.7066
Epoch 4/1000
30/30 [==============================] - 17s 550ms/step - loss: 0.0464 - accuracy: 0.7066
Epoch 5/1000
30/30 [==============================] - 16s 545ms/step - loss: 0.0462 - accuracy: 0.7075
Epoch 6/1000
30/30 [==============================] - 16s 542ms/step - loss: 0.0462 - accuracy: 0.7075
Epoch 7/1000
30/30 [==============================] - 16s 541ms/step - loss: 0.0774 - accuracy: 0.5774
Epoch 8/1000
30/30 [==============================] - 16s 539ms/step - loss: 0.0666 - accuracy: 0.6908
Epoch 9/1000
30/30 [==============================] - 16s 539ms/step - loss: 0.0528 - accuracy: 0.7066
Epoch 10/1000
30/30 [==============================] - 16s 538ms/step - l

In [ ]:
print(  f"Time_cnn_lstm: {Time_cnn_lstm}")

Time_cnn_lstm: 174.9189224243164


In [ ]:
loss_cnn_lstm, accuracy_cnn_lstm = model_cnn_lstm.evaluate(X_test, y_test)

68/68 [==============================] - 2s 22ms/step - loss: 0.0466 - accuracy: 0.7014


In [ ]:
model_cnn_lstm.save('cnn_lstm_vehicle_regressor.keras')